In [12]:
import pandas as pd

songs = pd.read_csv("data.tsv", sep="\t")

songs["Start"] = pd.to_timedelta(
    "00:" + songs["Start"].fillna("00:00").astype(str)
).dt.total_seconds()
songs["Start"] = songs["Start"].fillna(0)

songs["End"] = songs["End"].fillna(
    pd.to_datetime(songs["Start"] + 15, unit="s").dt.strftime("%M:%S")
)
songs["End"] = pd.to_timedelta(
    "00:" + songs["End"].fillna("00:00").astype(str)
).dt.total_seconds()


songs["R/O"] = songs["R/O"].fillna(0).astype(int).astype(str).str.zfill(2)
edition = int(songs["ED"].iloc[0])
isGF = songs.shape[0] < 30
#isGF = True

songs["SF"] = songs["SF"].fillna(0).astype(int)
songs["SF"] = "GF" if isGF else "SF" + songs["SF"].astype(str)
songs["SF"] = songs["SF"].replace({"SF0": "GF"})
sfs = list(songs["SF"].drop_duplicates())

songs["URL"] = songs["URL"].str.extract(
    r"(?<=be\/|\?v=)([a-zA-Z0-9_-]{11})", expand=False
)
urls = list(songs[songs["URL"].isna() == False]["URL"])
urls = pd.Series(
    songs[songs["URL"].isna() == False]["SF"].values,
    index=songs[songs["URL"].isna() == False]["URL"],
).to_dict()

songs = songs.sort_values(by=(["SF"] * (1 - isGF)) + ["R/O", "SF"])

In [32]:
import os
import yt_dlp
import skimage.filters
from moviepy import VideoFileClip, TextClip, ImageClip, ColorClip, CompositeVideoClip
from moviepy import vfx, afx
from moviepy.video.tools.subtitles import SubtitlesClip
from vtt_to_srt.vtt_to_srt import ConvertFile
from tqdm import tqdm


def blur(image): # TODO - FIX BLURS
    return skimage.filters.gaussian(image.astype(float), sigma=2)


def download_video(url, name, output_dir):
    output_path = os.path.join(output_dir, f"{name}.%(ext)s")
    ydl_opts = {
        "format": "bv*[height<=720]+ba/best[height<=720]/best",
        "outtmpl": output_path,
        #"download_archive": "download-archive.txt",
        "writesubtitles": True,
        "subtitleslangs": ["en.*"],
        "subtitlesformat": "vtt",
        "writeautomaticsub": False,
        "quiet": True,
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            info = ydl.extract_info(url, download=True)
            video_path = ydl.prepare_filename(info)
        except:
            print(f"Failed to download {name}")
        
    for fname in os.listdir("downloads"):
        if fname.startswith(f"{name}.en") and fname.endswith(".vtt"):
            if not os.path.exists("downloads/{name}.en.vtt"):
                os.rename(f"downloads/{fname}", f"downloads/{name}.en.vtt")
                ConvertFile(f"downloads/{name}.en.vtt", "utf-8").convert()
                os.remove(f"downloads/{name}.en.vtt")
    return video_path


def stack_clips(
    video_path,
    start,
    end,
    flag=False,
    ro="00",
    artist="",
    song_name="",
):

    def generate_text(
        text,
        size=70,
        color="white",
        duration=10,
        position=[0, 0],
        align="left",
        typeface="utility/font/NotoSans/NotoSans",
        weight="",
        style="Medium",
        format="ttf",
        shadow=False,
    ):
        txt_clip = (
            TextClip(
                text=str(text),
                font=f"{typeface}{weight}-{style}.{format}",
                font_size=size,
                color=color,
                horizontal_align=align,
                margin=(10, 10, 10, 10),
            )
            .with_duration(duration)
            .with_position(position)
        )
        if shadow:
            position = (position[0] + 2, position[1] + 2)
            return [
                generate_text(
                    text,
                    size,
                    "black",
                    duration,
                    position,
                    align,
                    typeface,
                    weight,
                    style,
                    format,
                    False,
                ),
                txt_clip
            ]
        return txt_clip

    def subtitle_generator(txt):
        replacements = {"â™ª": "", "\n": " ", "\r": ""}
        for old, new in replacements.items():
            txt = txt.replace(old, new).strip()
        return TextClip(
            text=str(txt),
            font=r"utility\font\NotoSans\NotoSans-Regular.ttf",
            font_size=30,
            color="white",
            stroke_color="black",
            stroke_width=2,
            text_align="center",
            horizontal_align="center",
            vertical_align="bottom",
            method="caption",
            margin=(10, 10, 10, 10),
            size=(1280, 700),
        )

    video = VideoFileClip(video_path)
    end = min(end, video.duration - 0.5)

    video = video.subclipped(max(start - 0.5, 0), end + 0.5).with_effects(
        [afx.AudioFadeIn(1), afx.AudioFadeOut(1), afx.AudioNormalize()]
    )

    clips = [ColorClip((1280, 720), (0, 0, 0), duration=video.duration)]

    width, height = video.w, video.h
    ratio = video.w / video.h
    if ratio > 1280 / 720:
        width = 120
    elif ratio > 950 / 720:
        height = 720

    if ratio <= 1.15:
        clips += [
            video.with_effects(
                [vfx.Resize(width=1280), vfx.Crop(y_center=height / 2, height=720)]
            ).without_audio()
        ]

    clips += [
        video.with_effects([vfx.Resize(width=width, height=height)]).with_position(
            "center"
        )
    ]

    if flag:
        try:
            clips.append(
                img_clip=ImageClip(
                    rf"utility\flags\{flag.upper()}.png",
                )
                .with_duration((end - start) + 1)
                .with_position((30, 30))
                .with_effects([vfx.Resize((80, 80))])  # Resize image
            )
        except:
            pass

    clips += (
        generate_text(
            ro,
            size=100,
            weight="_Condensed",
            style="Bold",
            align="center",
            position=(120, 20),
            duration=(end - start) + 1,
            shadow=True,
        )
        + generate_text(
            artist,
            size=30,
            style="Bold",
            position=(225, 20),
            duration=(end - start) + 1,
            shadow=True,
        )
        + generate_text(
            song_name,
            size=48,
            style="Medium",
            position=(225, 55),
            duration=(end - start) + 1,
            shadow=True,
        )
    )

    srt_path = f"{video_path[:-4]}.en.srt"
    if os.path.exists(srt_path):
        subtitles = SubtitlesClip(
            srt_path, make_textclip=subtitle_generator, encoding="utf8"
        ).subclipped(max(start - 0.5, 0), end + 0.5)
        clips.append(subtitles)

    return clips


def render_recap(clips, output_path):
    composite_clip = CompositeVideoClip(clips)
    composite_clip.write_videofile(
        output_path,
        threads=16,
        codec="libx264",
        ffmpeg_params=["-crf", "28"],
        logger=None,
    )
    return composite_clip


In [33]:
from slugify import slugify

clips = {}
output_dir = f"downloads"
clip_dir = f"e{edition}/clips/"
os.makedirs(output_dir, exist_ok=True)
os.makedirs(clip_dir, exist_ok=True)
for sf in sfs:
    os.makedirs(os.path.join(clip_dir, str(sf)), exist_ok=True)

for i, song in songs.iterrows():  # , total=sf_songs.shape[0]:
    base_name = slugify(
        f"{song['Artist']} {song['Song']}", lowercase=False, separator=" "
    )
    recap_path = os.path.join(
        clip_dir, 
        "gf" if (isGF) else str(song["SF"]), 
        f"{song['R/O']} {base_name}.mp4",
    )

    if os.path.isfile(recap_path):
        print(f"File already exists: {recap_path}")
    else:
        video_path = download_video(song["URL"], base_name, output_dir)
        clips = stack_clips(
            video_path,
            start=song["Start"],
            end=min(song["Start"] + 25, song["End"]),
            ro=song["R/O"],
            flag=song["Code"],
            artist=song["Artist"],
            song_name=song["Song"],
            
        )
        render_recap(clips, recap_path)

File already exists: e25/clips/GF\00 Luther Vandross Never Too Much.mp4
{'video_found': True, 'audio_found': True, 'metadata': {'COMPATIBLE_BRANDS': 'iso6av01mp41', 'MAJOR_BRAND': 'dash', 'MINOR_VERSION': '0', 'ENCODER': 'Lavf61.5.101'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1280, 720], 'bitrate': None, 'fps': 50.0, 'codec_name': 'av1', 'profile': '(libaom-av1)', 'metadata': {'Metadata': '', 'HANDLER_NAME': 'ISO Media file produced by Google Inc.', 'VENDOR_ID': '[0][0][0][0]', 'DURATION': '00:04:32.820000000'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': 'eng', 'default': True, 'fps': 48000, 'bitrate': None, 'metadata': {'Metadata': '', 'DURATION': '00:04:32.848000000'}}], 'input_number': 0}], 'duration': 272.85, 'bitrate': 1608, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'av1', 'video_profile': '(libaom

file being read: downloads/Weird Al Yankovic Amish Paradise.en.vtt

file created downloads/Weird Al Yankovic Amish Paradise.en.srt

{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2mp41', 'encoder': 'Lavf61.5.101'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [628, 480], 'bitrate': 190, 'fps': 29.97002997002997, 'codec_name': 'vp9', 'profile': '(Profile 0)', 'metadata': {'Metadata': '', 'handler_name': 'ISO Media file produced by Google Inc. Created on: 07/02/2025.', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': None, 'default': True, 'fps': 44100, 'bitrate': 127, 'metadata': {'Metadata': '', 'handler_name': 'SoundHandler', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 205.29, 'bitrate': 326, 'start': 0.0, 'default_video_input_number': 0, 

KeyboardInterrupt: 

In [6]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from tqdm import tqdm

# --- Authenticate ---
SCOPES = ["https://www.googleapis.com/auth/youtube"]
flow = InstalledAppFlow.from_client_secrets_file("utility/client_secrets.json", SCOPES)
credentials = flow.run_local_server(port=0)
youtube = build("youtube", "v3", credentials=credentials)

# --- Configuration ---
playlist_title = "My Playlist"
video_ids = urls

# --- Step 1: Search for existing playlist ---
def get_playlist_id_by_title(title):
    request = youtube.playlists().list(part="snippet", mine=True, maxResults=50)
    while request:
        response = request.execute()
        for item in response["items"]:
            if item["snippet"]["title"] == title:
                return item["id"]
        request = youtube.playlists().list_next(request, response)
    return None

playlist_id = {}
for sf in set(urls.values()):
    playlist_title = "JLSC E" + str(edition) + " " + sf
    playlist_id[sf] = get_playlist_id_by_title(playlist_title)

    if playlist_id[sf]:
        print(f"Found existing playlist: {playlist_id[sf]}, clearing it...")

        # List all items
        request = youtube.playlistItems().list(part="id", playlistId=playlist_id[sf], maxResults=50)
        while request:
            response = request.execute()
            for item in response["items"]:
                youtube.playlistItems().delete(id=item["id"]).execute()
            request = youtube.playlistItems().list_next(request, response)

    else:
        print("Playlist not found, creating new one...")
        response = youtube.playlists().insert(
            part="snippet,status",
            body={
                "snippet": {
                    "title": playlist_title,
                    "description": ""
                },
                "status": {
                    "privacyStatus": "unlisted"
                }
            }
        ).execute()
        playlist_id[sf] = response["id"]

# --- Step 3: Add videos ---
for video_id in tqdm(video_ids.keys()):
    youtube.playlistItems().insert(
        part="snippet",
        body={
            "snippet": {
                "playlistId": playlist_id[urls[video_id]],
                "resourceId": {
                    "kind": "youtube#video",
                    "videoId": video_id
                }
            }
        }
    ).execute()


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=1042242306104-u943shljfrbg68q34hlgu0j7av5b3df4.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A61563%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube&state=st5aHkFK2pWRknml2lntsPttjKELPn&access_type=offline
Playlist not found, creating new one...


100%|██████████| 26/26 [00:11<00:00,  2.24it/s]


In [38]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import configparser
import time
import csv

driver = webdriver.Chrome()
config = configparser.ConfigParser()
config.read(r'Utility\credentials.ini')
def scroll_click(element, driver=driver):
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", element)
    element.click()


def get_country_info(name, file_path="Utility\country_map.tsv"):
    with open(file_path, encoding="utf-8") as f:
        reader = csv.DictReader(f, delimiter="\t")
        for row in reader:
            if row["Country"].lower() == name.lower():
                return row["Full Name"], row["Scorewiz Code"]
    return None


scorewiz_username = config['scorewiz']['username']
scorewiz_password = config['scorewiz']['password']

driver.get("https://scorewiz.eu/login")

title = driver.title

driver.implicitly_wait(0.5)

driver.find_element(By.NAME, "email").send_keys(scorewiz_username)
driver.find_element(By.NAME, "pass").send_keys(scorewiz_password)
driver.find_element(By.CSS_SELECTOR, "#names > input:nth-child(2)").click() # login


scroll_click(driver.find_element(By.CLASS_NAME, "new-scoreboard"))
driver.find_element(By.NAME, "title").send_keys(time.time())
scroll_click(driver.find_element(By.ID, "votingMethodRsvp"))
scroll_click(driver.find_element(By.XPATH, './/input[@value="Create my scoreboard"]'))

scroll_click(driver.find_element(by=By.XPATH, value='.//section[@class="menu entries-juries"]/p/a'))


for i, song in songs.iterrows():
    country, country_code = get_country_info(song['Country'])
    print(country, country_code)
    song_name = (song['Artist']+" - "+song['Song']) or "Placeholder"
    submitter = song['Username']

    # Fill in flag field
    driver.find_element(By.ID, f"flag-select-{i+1}").send_keys(country[:2])

    scroll_click(driver.find_element(By.XPATH, f'//ul[contains(@class, "autocomplete-option-list")]/li[@data-value="{country_code}"]'))
    
    driver.find_element(By.ID, f"name{i+1}").send_keys(song_name)
    driver.find_element(By.ID, f"sponsor{i+1}").send_keys(submitter)

time.sleep(5)
scroll_click(driver.find_element(By.ID, 'namesSubmit'))

#driver.quit()

<>:17: SyntaxWarning: invalid escape sequence '\c'
<>:17: SyntaxWarning: invalid escape sequence '\c'
C:\Users\Henry\AppData\Local\Temp\ipykernel_91636\4102567660.py:17: SyntaxWarning: invalid escape sequence '\c'
  def get_country_info(name, file_path="Utility\country_map.tsv"):


['scorewiz']
Puerto Rico pr


C:\Users\Henry\AppData\Local\Temp\ipykernel_91636\4102567660.py:17: SyntaxWarning: invalid escape sequence '\c'
  def get_country_info(name, file_path="Utility\country_map.tsv"):


AttributeError: 'NoneType' object has no attribute 'send_keys'